In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
df=pd.read_csv('file-path/data/diabetes_feature.csv',encoding='cp949')
df.shape

(768, 16)

In [8]:
df_insulin = pd.read_csv('file-path/data/diabetes_fill_insulin.csv')
df['Insulin'] = df_insulin['Insulin']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,182.665610,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,57.733250,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,186.519678,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.000000,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.000000,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [9]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [10]:
x=df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
      'Insulin']]
x.shape

(768, 8)

In [12]:
y=df['Outcome']
y.shape

(768,)

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
x, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier


estimators = [DecisionTreeClassifier(random_state=42),
             RandomForestClassifier(random_state=42),
             GradientBoostingClassifier(random_state=42)]
estimators


[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [17]:

from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions={'max_depth':max_depth,
                     'max_features':max_features}

results=[]
for estimator in estimators:
    result=[]
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions['n_estimators'] = np.random.randint(100,200,10)
    clf = RandomizedSearchCV(estimator,
                       param_distributions,
                       n_iter=100,
                       scoring='accuracy',
                       n_jobs=-1,
                       cv=5,
                       verbose=2)

    clf.fit(x_train,y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(x_test,y_test))
    result.append(clf.cv_results_)
    results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   41.6s finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.6min finished


In [18]:

df = pd.DataFrame(results,
             columns=['estimator','best_params','train_score','test_score','cv_result'])
df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.6276496645010092, 'max_dept...",0.744329,0.727273,"{'mean_fit_time': [0.005488157272338867, 0.006..."
1,RandomForestClassifier,"{'n_estimators': 196, 'max_features': 0.800773...",0.794842,0.733766,"{'mean_fit_time': [0.3477972984313965, 0.28114..."
2,GradientBoostingClassifier,"{'n_estimators': 167, 'max_features': 0.555631...",0.789951,0.746753,"{'mean_fit_time': [0.23317427635192872, 0.1855..."


In [ ]:
pd.DataFrame(df.loc[1,'cv_result']).sort_values(by='rank_test_score')

In [ ]:
# 인슐린을 회귀로 예측했을떄 정확도가 오히려 조김더 떨어졌다.